# Outil pour analyse des fichiers de sortie

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

## 0. Format du fichier

### 0.1 Visulation des données

In [ ]:
trip_info_analyses_0 = pd.read_xml("C:/Users/eleon/OneDrive - entpe.fr/Documents/these/ITS/ITS/ITS/Projet_09_VSL/stats/tripinfo.xml")


In [ ]:
trip_info_analyses_0.head()

In [ ]:
print(trip_info_analyses_0.columns)

In [ ]:
trip_info_analyses_0.duration.describe()

In [ ]:
trip_info_analyses_0.vType

### 0.2 Tri des données

In [ ]:
trip_info_tri = trip_info_analyses_0[trip_info_analyses_0.depart > 300]
number_rows = len(trip_info_analyses_0.index)
print(number_rows)
number_rows_tri = len(trip_info_tri.index)
print(number_rows_tri)

In [ ]:
trip_info_tri_2 = trip_info_tri[trip_info_tri.vType != "autre"]
print(len(trip_info_tri_2.index))

In [ ]:
trip_info_tri_2.duration.describe()

In [ ]:
def tri_colonne(path):
    trip_info = pd.read_xml(path)
    trip_info_tri = trip_info[trip_info.depart > 300]
    trip_info_tri_2 = trip_info_tri[trip_info_tri.vType != "autre"]
    return trip_info_tri_2
    

In [ ]:
path = "C:/Users/eleon/OneDrive - entpe.fr/Documents/these/ITS/ITS/ITS/Projet_09_VSL/stats/tripinfo.xml"
trip_info_0 = tri_colonne(path)

## 1. Exemples d'indicateurs et d'outils pour observer l'efficacité du réseau

### 1.1 Indicateur: temps de parcours

In [ ]:
### box plot

fig = go.Figure()
fig.add_trace(go.Box(y=trip_info_0["duration"], name = "Baseline"))
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=trip_info_0["duration"], name = "Baseline"))
fig.show()

In [ ]:
### histogramme

fig = go.Figure()
fig.add_trace(go.Histogram(x=trip_info_0["duration"], name = "Baseline"))
fig.update_layout(barmode = "overlay")
fig.update_traces(opacity = 0.5)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(y=trip_info_0["duration"], name = "Baseline"))
fig.update_layout(barmode = "overlay")
fig.update_traces(opacity = 0.5)
fig.show()

In [ ]:
print(trip_info_0.duration.describe())

In [ ]:
print('Total Time Spent (h)', trip_info_0["duration"].sum()/3600)


In [ ]:
ar = np.array([[trip_info_0["duration"].sum()/3600]])
df = pd.DataFrame(ar, index = ['Baseline'], columns = ['Total Time Spent '])
print(df)   

### 1.2 Indicateur: vitesse moyenne

In [ ]:
path_traj = "C:/Users/eleon/OneDrive - entpe.fr/Documents/these/ITS/ITS/ITS/Projet_09_VSL/stats/trajectoriesStats.xml"
traj_analyses_0 = pd.read_xml(path_traj)
traj_analyses_0.head()
## nécessaire de convertir xml en csv

In [ ]:
path_fcd_csv = "C:/Users/eleon/OneDrive - entpe.fr/Documents/these/ITS/ITS/ITS/Projet_09_VSL/stats/fcdStats.csv"
fcd_analyses_0 = pd.read_csv(path_fcd_csv, sep = ';')
fcd_analyses_0.head()

In [ ]:
print(fcd_analyses_0.columns)

In [ ]:
fcd_0_tri = fcd_analyses_0[fcd_analyses_0.timestep_time > 300]
number_rows = len(fcd_analyses_0.index)
print(number_rows)
number_rows_tri = len(fcd_0_tri.index)
print(number_rows_tri)

fcd_0_tri_2 = fcd_0_tri[fcd_0_tri.vehicle_type != "autre"]
print(len(fcd_0_tri_2.index))

In [ ]:
## Groupe les vehicles dans le même pas de temps et fait la moyenne
fcd_0 = fcd_0_tri_2.groupby('timestep_time',as_index=False)
fcd_mean_0=fcd_0.mean()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=fcd_mean_0["timestep_time"].tolist(), y=fcd_mean_0["vehicle_speed"].tolist() ,name = 'Baseline mean speed'))

fig.show()

### 1.3 Indicateur: trajectoire des véhicules

In [ ]:
#det
print(len(fcd_0_tri_2.index))
index_with_nan = fcd_0_tri_2.index[fcd_0_tri_2.isnull().any(axis=1)]
fcd_0_tri_2.drop(index_with_nan,0, inplace=True)
print(len(fcd_0_tri_2.index))

fig = px.scatter(fcd_0_tri_2, x='timestep_time', y='vehicle_pos', color='vehicle_speed', color_continuous_scale='jet_r' )
fig.show()

## 2. Exemples d'indicateurs pour observer l'impact environnmental du réseau

### 2.1 Indicateur: émissions CO2

In [ ]:
path_tripinfo_csv = "C:/Users/eleon/OneDrive - entpe.fr/Documents/these/ITS/ITS/ITS/Projet_09_VSL/stats/tripinfo.csv"
trip_csv_0 = pd.read_csv(path_tripinfo_csv, sep = ';')
print(trip_csv_0.head())

In [ ]:
print(trip_csv_0.columns)

In [ ]:
print(len(trip_csv_0.index))
trip_tri_1 = trip_csv_0[trip_csv_0.tripinfo_depart > 300]
print(len(trip_tri_1))
trip_tri_2 = trip_tri_1[trip_tri_1.tripinfo_vType != "autre"]
print(len(trip_tri_2))

In [ ]:
trip_tri_2.emissions_CO2_abs.describe()

In [ ]:
trip_tri_2.emissions_CO2_abs.sum()

## 3. Exemples d'indicateurs pour observer la sécurité sur le réseau

### 3.1 Indicateur: homogénéisation des vitesses